# 📊 Insurance Cost Prediction — Data Analytics Using Python

## Mini Project Overview

This notebook demonstrates a complete data analytics workflow for predicting insurance charges based on demographic and health factors. We will explore the data, perform statistical analysis, and build a predictive model using Linear Regression.


## Step 1: Problem Definition & Dataset Selection

### Objective

The goal of this project is to **predict insurance charges** based on various demographic and health factors such as age, BMI, smoking status, region, and number of children.

### Real-World Relevance

Insurance companies need to accurately predict healthcare costs to:
- Set appropriate premium rates
- Assess risk for different customer segments
- Make data-driven pricing decisions
- Understand which factors most influence medical expenses

This analysis helps both insurers and policyholders understand what drives healthcare costs.

### Dataset Source

**Medical Cost Personal Dataset** from Kaggle

- **URL**: https://www.kaggle.com/datasets/mirichoi0218/insurance
- This dataset contains information about individuals and their insurance charges


In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Setting style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [6]:
# !pip install matplotlib
!pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Loading the dataset
# Note: You may need to download the dataset from Kaggle and update the path
# For this example, we'll assume the file is named 'insurance.csv' in the same directory
df = pd.read_csv('insurance.csv')

# Displaying first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Getting descriptive statistics
df.describe()

**Explanation of df.describe() output:**
- **count**: Number of non-null values
- **mean**: Average value
- **std**: Standard deviation (measure of spread)
- **min**: Minimum value
- **25%**: First quartile (25% of values are below this)
- **50%**: Median (middle value)
- **75%**: Third quartile (75% of values are below this)
- **max**: Maximum value

This helps us understand the distribution and range of numerical variables.


### Types of Variables

**Numerical Variables:**
- `age`: Age of the person (continuous)
- `bmi`: Body Mass Index (continuous)
- `children`: Number of children/dependents (discrete)
- `charges`: Insurance charges in dollars (continuous, our target variable)

**Categorical Variables:**
- `sex`: Gender (male/female)
- `smoker`: Smoking status (yes/no)
- `region`: Geographic region (northeast/northwest/southeast/southwest)

## Step 2: Data Cleaning & Preparation

Data cleaning is crucial to ensure our model learns from accurate and complete data. Let's check for issues and prepare our dataset for analysis.

In [ ]:
# Checking missing values to ensure data completeness
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)
print(f"\nTotal missing values: {missing_values.sum()}")

**Result Explanation:**
If all columns show 0 missing values, the dataset is complete. Missing values can cause errors in modeling, so it's important to check this first.

In [ ]:
# Removing duplicates to avoid biased model training
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

if duplicates > 0:
    df = df.drop_duplicates()
    print(f"After removing duplicates: {df.shape[0]} rows remaining")
else:
    print("No duplicates found. Dataset is clean.")

**Why remove duplicates?**
Duplicate rows can make our model overfit to repeated patterns and give us an overly optimistic view of model performance.


In [ ]:
# Detecting outliers using IQR (Interquartile Range) method on BMI and Charges
# Outliers are extreme values that might skew our analysis

def detect_outliers_iqr(data, column):
    """Detect outliers using IQR method"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Checking outliers in BMI
bmi_outliers, bmi_lower, bmi_upper = detect_outliers_iqr(df, 'bmi')
print(f"BMI Outliers:")
print(f"Lower bound: {bmi_lower:.2f}, Upper bound: {bmi_upper:.2f}")
print(f"Number of outliers: {len(bmi_outliers)}")

# Checking outliers in Charges
charges_outliers, charges_lower, charges_upper = detect_outliers_iqr(df, 'charges')
print(f"\nCharges Outliers:")
print(f"Lower bound: ${charges_lower:.2f}, Upper bound: ${charges_upper:.2f}")
print(f"Number of outliers: {len(charges_outliers)}")

**Outlier Detection Explanation:**
- IQR (Interquartile Range) = Q3 - Q1
- Values outside [Q1 - 1.5×IQR, Q3 + 1.5×IQR] are considered outliers
- For insurance charges, outliers might represent legitimate high-cost cases (e.g., serious illnesses)
- We'll keep outliers for now as they may contain important information


In [ ]:
# Encoding categorical variables using One-Hot Encoding
# This converts categorical data into numerical format that machine learning models can understand

# Creating a copy to preserve original data
df_encoded = df.copy()

# One-Hot Encoding for categorical variables
df_encoded = pd.get_dummies(df_encoded, columns=['sex', 'smoker', 'region'], drop_first=True)

# Displaying the encoded dataset
print("Dataset after encoding:")
print(df_encoded.head())
print(f"\nNew shape: {df_encoded.shape}")

**One-Hot Encoding Explanation:**
- Converts each category into a binary (0/1) column
- `drop_first=True` removes one column per category to avoid multicollinearity
- Example: `smoker` becomes `smoker_yes` (1 if smoker, 0 if not)
- This allows the model to understand categorical relationships numerically


In [ ]:
# Preparing features and target variable
# Separating independent variables (features) and dependent variable (target)

X = df_encoded.drop('charges', axis=1)  # Features (all columns except charges)
y = df_encoded['charges']  # Target variable (what we want to predict)

print("Features (X):")
print(X.columns.tolist())
print(f"\nTarget (y): charges")
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

**Note on Scaling:**
For Linear Regression, scaling is optional but can help with interpretation. Since we're using Linear Regression, we'll proceed without scaling. If we were using algorithms like KNN or SVM, scaling would be essential.


## Step 3: Exploratory Data Analysis (EDA)

Exploratory Data Analysis helps us understand patterns, relationships, and distributions in our data before building models. It's like getting to know your data before making predictions.


### Univariate Analysis

Univariate analysis examines one variable at a time to understand its distribution.


In [ ]:
# Histogram of Age
plt.figure(figsize=(10, 6))
plt.hist(df['age'], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
plt.title('Distribution of Age', fontsize=16, fontweight='bold')
plt.xlabel('Age (years)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

**Age Distribution Insights:**
The histogram shows how ages are distributed across the dataset. We can see if the data is evenly spread or concentrated in certain age groups.


In [ ]:
# Histogram of BMI
plt.figure(figsize=(10, 6))
plt.hist(df['bmi'], bins=20, color='lightgreen', edgecolor='black', alpha=0.7)
plt.title('Distribution of BMI (Body Mass Index)', fontsize=16, fontweight='bold')
plt.xlabel('BMI', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.axvline(df['bmi'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["bmi"].mean():.2f}')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

**BMI Distribution Insights:**
BMI typically ranges from 18.5 (underweight) to 30+ (obese). This histogram shows the distribution of BMI values in our dataset.


In [ ]:
# Boxplot of Charges
plt.figure(figsize=(10, 6))
plt.boxplot(df['charges'], vert=True, patch_artist=True,
            boxprops=dict(facecolor='lightcoral', alpha=0.7),
            medianprops=dict(color='black', linewidth=2))
plt.title('Distribution of Insurance Charges', fontsize=16, fontweight='bold')
plt.ylabel('Charges ($)', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

**Charges Boxplot Insights:**
- The box shows the interquartile range (middle 50% of data)
- The line inside the box is the median
- Whiskers extend to show the range (excluding outliers)
- Points beyond whiskers are outliers
- This plot reveals if charges are skewed or have extreme values


### Bivariate Analysis

Bivariate analysis examines relationships between two variables, helping us understand how features relate to insurance charges.


In [ ]:
# Age vs Charges Scatter Plot
plt.figure(figsize=(10, 6))
plt.scatter(df['age'], df['charges'], alpha=0.5, color='blue', s=50)
plt.title('Age vs Insurance Charges', fontsize=16, fontweight='bold')
plt.xlabel('Age (years)', fontsize=12)
plt.ylabel('Charges ($)', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**Age vs Charges Insights:**
This scatter plot shows if there's a relationship between age and insurance charges. Generally, older people tend to have higher medical costs.


In [ ]:
# BMI vs Charges Scatter Plot
plt.figure(figsize=(10, 6))
plt.scatter(df['bmi'], df['charges'], alpha=0.5, color='green', s=50)
plt.title('BMI vs Insurance Charges', fontsize=16, fontweight='bold')
plt.xlabel('BMI', fontsize=12)
plt.ylabel('Charges ($)', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**BMI vs Charges Insights:**
Higher BMI often correlates with health issues, which may lead to higher insurance charges. This plot helps us see if that relationship exists in our data.


In [ ]:
# Boxplot: Smoker vs Charges
plt.figure(figsize=(10, 6))
df.boxplot(column='charges', by='smoker', figsize=(10, 6), 
           patch_artist=True,
           boxprops=dict(facecolor='lightblue', alpha=0.7))
plt.title('Insurance Charges: Smokers vs Non-Smokers', fontsize=16, fontweight='bold')
plt.suptitle('')  # Remove default title
plt.xlabel('Smoker Status', fontsize=12)
plt.ylabel('Charges ($)', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**Smoker vs Charges Insights:**
This comparison is crucial! Smokers typically have much higher insurance charges due to increased health risks. The boxplot clearly shows the difference between the two groups.


### Multivariate Analysis

Multivariate analysis examines relationships between multiple variables simultaneously.


In [ ]:
# Correlation Heatmap
# Selecting only numerical columns for correlation
numerical_cols = ['age', 'bmi', 'children', 'charges']
correlation_matrix = df[numerical_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap of Numerical Variables', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

**Correlation Heatmap Insights:**
- Values range from -1 to +1
- +1 means perfect positive correlation (both increase together)
- -1 means perfect negative correlation (one increases, other decreases)
- 0 means no linear relationship
- Darker colors indicate stronger relationships
- This helps identify which numerical features are most related to charges


In [ ]:
# Pairplot (optional but informative)
# Shows pairwise relationships between numerical variables
sns.pairplot(df[numerical_cols], diag_kind='hist', height=2.5)
plt.suptitle('Pairplot of Numerical Variables', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

**Pairplot Insights:**
The pairplot shows all combinations of scatter plots and histograms. The diagonal shows distributions, while off-diagonal plots show relationships between pairs of variables.


In [ ]:
# Descriptive Statistics for Charges
print("Descriptive Statistics for Insurance Charges:")
print("=" * 50)
print(f"Mean: ${df['charges'].mean():.2f}")
print(f"Median: ${df['charges'].median():.2f}")
print(f"Standard Deviation: ${df['charges'].std():.2f}")
print(f"Minimum: ${df['charges'].min():.2f}")
print(f"Maximum: ${df['charges'].max():.2f}")

# Skewness and Kurtosis
from scipy.stats import skew, kurtosis
print(f"\nSkewness: {skew(df['charges']):.2f}")
print(f"Kurtosis: {kurtosis(df['charges']):.2f}")

**Statistical Measures Explanation:**
- **Mean**: Average value
- **Median**: Middle value (less affected by outliers)
- **Standard Deviation**: Measure of spread/variability
- **Skewness**: Measures asymmetry (positive = right-skewed, negative = left-skewed)
- **Kurtosis**: Measures tail heaviness (high kurtosis = more outliers)


### EDA Summary

**What patterns do we observe?**
1. **Smoking Status**: Smokers have significantly higher charges than non-smokers
2. **Age**: There appears to be a positive relationship between age and charges
3. **BMI**: Higher BMI may correlate with higher charges, especially when combined with other factors
4. **Distribution**: Charges are right-skewed (many low values, few very high values)

**Why EDA matters?**
- Helps us understand our data before modeling
- Identifies relationships and patterns
- Reveals outliers and data quality issues
- Guides feature selection and model choice

**Which variables seem to influence charges?**
Based on our visualizations:
- **Smoker status** appears to be the strongest factor
- **Age** shows a clear positive trend
- **BMI** may have an impact, especially at higher values
- **Region** and **sex** may have smaller effects


## Step 4: Statistical Analysis & Hypothesis Testing

Statistical testing helps us make data-driven conclusions about relationships in our data. We'll test whether smokers have significantly higher insurance charges than non-smokers.


### Hypothesis

- **H₀ (Null Hypothesis)**: Smokers and non-smokers have equal mean charges
- **H₁ (Alternative Hypothesis)**: Smokers have significantly higher mean charges

We'll use a two-sample t-test to test this hypothesis.


In [ ]:
# Separating smokers and non-smokers groups
smokers_charges = df[df['smoker'] == 'yes']['charges']
non_smokers_charges = df[df['smoker'] == 'no']['charges']

print("Group Statistics:")
print("=" * 50)
print(f"Smokers - Count: {len(smokers_charges)}")
print(f"Smokers - Mean: ${smokers_charges.mean():.2f}")
print(f"Smokers - Std: ${smokers_charges.std():.2f}")
print(f"\nNon-Smokers - Count: {len(non_smokers_charges)}")
print(f"Non-Smokers - Mean: ${non_smokers_charges.mean():.2f}")
print(f"Non-Smokers - Std: ${non_smokers_charges.std():.2f}")

In [ ]:
# Running Independent two-sample t-test
# This tests if the means of two independent groups are significantly different
t_statistic, p_value = stats.ttest_ind(smokers_charges, non_smokers_charges)

print("T-Test Results:")
print("=" * 50)
print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.2e}")

# Significance level
alpha = 0.05
print(f"\nSignificance level (α): {alpha}")

if p_value < alpha:
    print(f"\nResult: Reject H₀ (p-value {p_value:.2e} < {alpha})")
    print("Conclusion: Smokers have significantly higher mean charges than non-smokers.")
else:
    print(f"\nResult: Fail to reject H₀ (p-value {p_value:.2e} >= {alpha})")
    print("Conclusion: No significant difference in mean charges between groups.")

In [ ]:
# Calculating Confidence Interval for the difference in means
# 95% confidence interval
confidence_level = 0.95
degrees_of_freedom = len(smokers_charges) + len(non_smokers_charges) - 2

mean_diff = smokers_charges.mean() - non_smokers_charges.mean()
std_error = np.sqrt((smokers_charges.std()**2 / len(smokers_charges)) + 
                    (non_smokers_charges.std()**2 / len(non_smokers_charges)))

t_critical = stats.t.ppf((1 + confidence_level) / 2, degrees_of_freedom)
margin_of_error = t_critical * std_error

ci_lower = mean_diff - margin_of_error
ci_upper = mean_diff + margin_of_error

print("Confidence Interval:")
print("=" * 50)
print(f"Mean difference (Smokers - Non-smokers): ${mean_diff:.2f}")
print(f"95% Confidence Interval: [${ci_lower:.2f}, ${ci_upper:.2f}]")
print(f"\nInterpretation: We are 95% confident that the true difference in mean charges")
print(f"between smokers and non-smokers lies between ${ci_lower:.2f} and ${ci_upper:.2f}.")

### Understanding P-Value

**What is a p-value?**
- The p-value tells us the probability of observing our results (or more extreme) if the null hypothesis were true
- A small p-value (< 0.05) suggests that our observed difference is unlikely to occur by chance
- If p-value < 0.05, we reject H₀ and conclude there is a significant difference

**In our case:**
- If p-value is very small (e.g., < 0.001), it strongly suggests smokers have higher charges
- The smaller the p-value, the stronger the evidence against the null hypothesis

### Type I and Type II Errors

**Type I Error (False Positive):**
- Rejecting H₀ when it's actually true
- Example: Concluding smokers have higher charges when they actually don't
- Probability = α (significance level, usually 0.05)

**Type II Error (False Negative):**
- Failing to reject H₀ when it's actually false
- Example: Concluding no difference when smokers actually do have higher charges
- Probability = β (harder to control, depends on sample size and effect size)

In simple words: Type I is a "false alarm" - we think we found something that isn't there. Type II is a "missed opportunity" - we failed to detect something that is actually there.


## Step 5: Modeling (Linear Regression)

Now we'll build a Linear Regression model to predict insurance charges based on all available features. Linear Regression finds the best line (or hyperplane) that fits our data.


In [ ]:
# Splitting data into training and testing sets (80/20 split)
# This allows us to train on one portion and test on unseen data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data Split:")
print("=" * 50)
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"Number of features: {X_train.shape[1]}")

In [ ]:
# Fitting the Linear Regression model
# The model learns the relationship between features and target variable
model = LinearRegression()
model.fit(X_train, y_train)

print("Model trained successfully!")
print(f"Number of coefficients: {len(model.coef_)}")
print(f"Intercept: ${model.intercept_:.2f}")

In [ ]:
# Making predictions on test set
y_pred = model.predict(X_test)

print("Sample Predictions:")
print("=" * 50)
comparison_df = pd.DataFrame({
    'Actual': y_test.values[:10],
    'Predicted': y_pred[:10],
    'Difference': y_test.values[:10] - y_pred[:10]
})
comparison_df.index = range(1, 11)
print(comparison_df.round(2))


In [ ]:
# Evaluating performance using RMSE, MAE, and R² Score
# RMSE: Root Mean Squared Error (lower is better, in same units as target)
# MAE: Mean Absolute Error (lower is better, in same units as target)
# R²: Coefficient of Determination (higher is better, 0-1 scale)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Performance Metrics:")
print("=" * 50)
print(f"R² Score: {r2:.4f}")
print(f"RMSE: ${rmse:.2f}")
print(f"MAE: ${mae:.2f}")

print("\nInterpretation:")
print(f"- R² of {r2:.2%} means the model explains {r2:.2%} of variance in charges")
print(f"- RMSE of ${rmse:.2f} means predictions are off by about ${rmse:.2f} on average")
print(f"- MAE of ${mae:.2f} means average absolute error is ${mae:.2f}")

In [ ]:
# Displaying coefficients and their interpretation
# Coefficients show how much each feature affects the target variable
coefficients_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})
coefficients_df = coefficients_df.sort_values('Coefficient', key=abs, ascending=False)

print("Feature Coefficients (sorted by absolute value):")
print("=" * 50)
print(coefficients_df.to_string(index=False))

print(f"\nIntercept: ${model.intercept_:.2f}")
print("\nInterpretation:")
print("- Positive coefficients increase charges")
print("- Negative coefficients decrease charges")
print("- Larger absolute values indicate stronger influence")

### Coefficient Interpretation

**Which features increase charges?**
- Features with positive coefficients increase insurance charges
- The feature with the largest positive coefficient has the strongest positive impact

**Impact of Smoking:**
- The `smoker_yes` coefficient shows how much more smokers pay compared to non-smokers
- This should be a large positive value, confirming our hypothesis test results

**Impact of BMI and Age:**
- BMI and Age coefficients show their individual contributions
- Positive values mean higher BMI/age leads to higher charges
- The magnitude shows the strength of the relationship


In [ ]:
# Residual Plot
# Residuals = Actual - Predicted
# Good model should have residuals randomly scattered around zero
residuals = y_test - y_pred

plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.5, color='purple')
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.title('Residual Plot', fontsize=16, fontweight='bold')
plt.xlabel('Predicted Charges ($)', fontsize=12)
plt.ylabel('Residuals (Actual - Predicted) ($)', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**Residual Plot Interpretation:**
- Residuals should be randomly scattered around zero (red line)
- No clear patterns indicate a good model fit
- Patterns (curves, funnels) suggest the model might need improvement


In [ ]:
# Predicted vs Actual Charges
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5, color='blue', s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', linewidth=2, label='Perfect Prediction Line')
plt.title('Predicted vs Actual Insurance Charges', fontsize=16, fontweight='bold')
plt.xlabel('Actual Charges ($)', fontsize=12)
plt.ylabel('Predicted Charges ($)', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---

## Step 5.5: Improved Linear Regression Model (Feature Engineering)

The basic model achieved ~80% R². Let's improve it by adding:
- **Interaction terms** (e.g., smoker × age, smoker × bmi) - captures how variables work together
- **Polynomial features** (e.g., age², bmi²) - captures non-linear relationships
- **Better feature engineering** - creates more informative predictors

This is still Linear Regression, just with better features!


In [ ]:
# Creating an improved feature set with feature engineering
# We'll add interaction terms and polynomial features to capture non-linear relationships

# Starting with the original encoded dataset
df_improved = df_encoded.copy()

# Creating interaction terms - these capture how variables work together
# Example: Smoker × Age means the effect of age is different for smokers vs non-smokers
df_improved['smoker_age'] = df_improved['age'] * df_improved['smoker_yes']
df_improved['smoker_bmi'] = df_improved['bmi'] * df_improved['smoker_yes']
df_improved['age_bmi'] = df_improved['age'] * df_improved['bmi']
df_improved['age_children'] = df_improved['age'] * df_improved['children']

# Creating polynomial features - captures non-linear relationships
# Age squared: older people might have exponentially higher costs
df_improved['age_squared'] = df_improved['age'] ** 2
# BMI squared: very high BMI might have disproportionate impact
df_improved['bmi_squared'] = df_improved['bmi'] ** 2

# Creating BMI categories as features (BMI > 30 is obese, which might have extra cost)
df_improved['bmi_obese'] = (df_improved['bmi'] > 30).astype(int)
df_improved['bmi_overweight'] = ((df_improved['bmi'] > 25) & (df_improved['bmi'] <= 30)).astype(int)

# Age groups - older people might have different cost structures
df_improved['age_old'] = (df_improved['age'] > 50).astype(int)
df_improved['age_middle'] = ((df_improved['age'] > 30) & (df_improved['age'] <= 50)).astype(int)

print("New features created:")
print("=" * 50)
new_features = ['smoker_age', 'smoker_bmi', 'age_bmi', 'age_children', 
                'age_squared', 'bmi_squared', 'bmi_obese', 'bmi_overweight',
                'age_old', 'age_middle']
print(f"Added {len(new_features)} new engineered features")
print(f"Total features now: {df_improved.shape[1] - 1}")  # -1 for target variable


In [ ]:
# Preparing improved features and target
X_improved = df_improved.drop('charges', axis=1)
y_improved = df_improved['charges']

# Splitting the improved dataset
X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(
    X_improved, y_improved, test_size=0.2, random_state=42
)

print("Improved Dataset Split:")
print("=" * 50)
print(f"Training set size: {X_train_imp.shape[0]} samples")
print(f"Test set size: {X_test_imp.shape[0]} samples")
print(f"Number of features: {X_train_imp.shape[1]} (increased from {X_train.shape[1]})")


In [ ]:
# Training the improved Linear Regression model
model_improved = LinearRegression()
model_improved.fit(X_train_imp, y_train_imp)

print("Improved model trained successfully!")
print(f"Number of coefficients: {len(model_improved.coef_)}")
print(f"Intercept: ${model_improved.intercept_:.2f}")


In [ ]:
# Making predictions with improved model
y_pred_improved = model_improved.predict(X_test_imp)

# Evaluating improved model performance
rmse_improved = np.sqrt(mean_squared_error(y_test_imp, y_pred_improved))
mae_improved = mean_absolute_error(y_test_imp, y_pred_improved)
r2_improved = r2_score(y_test_imp, y_pred_improved)

print("IMPROVED Model Performance Metrics:")
print("=" * 50)
print(f"R² Score: {r2_improved:.4f}")
print(f"RMSE: ${rmse_improved:.2f}")
print(f"MAE: ${mae_improved:.2f}")

print("\nCOMPARISON:")
print("=" * 50)
print(f"{'Metric':<15} {'Basic Model':<15} {'Improved Model':<15} {'Improvement':<15}")
print("-" * 60)
print(f"{'R² Score':<15} {r2:.4f}         {r2_improved:.4f}         {((r2_improved - r2) * 100):+.2f}%")
print(f"{'RMSE':<15} ${rmse:.2f}        ${rmse_improved:.2f}        {((rmse - rmse_improved) / rmse * 100):+.2f}%")
print(f"{'MAE':<15} ${mae:.2f}        ${mae_improved:.2f}        {((mae - mae_improved) / mae * 100):+.2f}%")

print(f"\nR² Improvement: {((r2_improved - r2) * 100):.2f} percentage points")
print(f"RMSE Reduction: {((rmse - rmse_improved) / rmse * 100):.2f}%")
print(f"MAE Reduction: {((mae - mae_improved) / mae * 100):.2f}%")


In [ ]:
# Displaying top features from improved model
coefficients_improved = pd.DataFrame({
    'Feature': X_improved.columns,
    'Coefficient': model_improved.coef_
})
coefficients_improved = coefficients_improved.sort_values('Coefficient', key=abs, ascending=False)

print("Top 15 Most Important Features (Improved Model):")
print("=" * 60)
print(coefficients_improved.head(15).to_string(index=False))


**Key Improvements Explained:**

1. **Interaction Terms**: 
   - `smoker_age`: Captures that age affects smokers differently than non-smokers
   - `smoker_bmi`: BMI might have different impact for smokers
   - `age_bmi`: Age and BMI together might have combined effects

2. **Polynomial Features**:
   - `age_squared`: Captures that costs might increase exponentially with age
   - `bmi_squared`: Very high BMI might have disproportionate cost impact

3. **Categorical Features**:
   - `bmi_obese`, `bmi_overweight`: Captures threshold effects
   - `age_old`, `age_middle`: Captures age group effects

These features help the model capture non-linear relationships while still using Linear Regression!


In [ ]:
# Comparing predictions: Basic vs Improved Model
# Note: Both models use same random_state=42, so test sets should align
# But to be safe, we'll compare on the improved model's test set
comparison_improved = pd.DataFrame({
    'Actual': y_test_imp.values[:10],
    'Improved Model': y_pred_improved[:10],
    'Improved Error': abs(y_test_imp.values[:10] - y_pred_improved[:10])
})
comparison_improved.index = range(1, 11)

print("Sample Predictions (Improved Model):")
print("=" * 60)
print(comparison_improved.round(2))
print(f"\nAverage Absolute Error: ${comparison_improved['Improved Error'].mean():.2f}")
print(f"Median Absolute Error: ${comparison_improved['Improved Error'].median():.2f}")

In [ ]:
# Improved Model: Predicted vs Actual Plot
plt.figure(figsize=(12, 5))

# Subplot 1: Basic Model
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred, alpha=0.5, color='blue', s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', linewidth=2, label='Perfect Prediction')
plt.title(f'Basic Model (R² = {r2:.3f})', fontsize=14, fontweight='bold')
plt.xlabel('Actual Charges ($)', fontsize=11)
plt.ylabel('Predicted Charges ($)', fontsize=11)
plt.legend()
plt.grid(alpha=0.3)

# Subplot 2: Improved Model
plt.subplot(1, 2, 2)
plt.scatter(y_test_imp, y_pred_improved, alpha=0.5, color='green', s=50)
plt.plot([y_test_imp.min(), y_test_imp.max()], [y_test_imp.min(), y_test_imp.max()], 
         'r--', linewidth=2, label='Perfect Prediction')
plt.title(f'Improved Model (R² = {r2_improved:.3f})', fontsize=14, fontweight='bold')
plt.xlabel('Actual Charges ($)', fontsize=11)
plt.ylabel('Predicted Charges ($)', fontsize=11)
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Improved Model: Residual Plot
residuals_improved = y_test_imp - y_pred_improved

plt.figure(figsize=(12, 5))

# Subplot 1: Basic Model Residuals
plt.subplot(1, 2, 1)
plt.scatter(y_pred, residuals, alpha=0.5, color='purple')
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.title('Basic Model Residuals', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Charges ($)', fontsize=11)
plt.ylabel('Residuals ($)', fontsize=11)
plt.grid(alpha=0.3)

# Subplot 2: Improved Model Residuals
plt.subplot(1, 2, 2)
plt.scatter(y_pred_improved, residuals_improved, alpha=0.5, color='orange')
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.title('Improved Model Residuals', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Charges ($)', fontsize=11)
plt.ylabel('Residuals ($)', fontsize=11)
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Why Feature Engineering Improved the Model

**The improved model achieves higher R² because:**

1. **Interaction Terms**: Capture how variables work together (e.g., smoking + age = much higher cost)
2. **Polynomial Features**: Capture non-linear relationships (e.g., costs increase faster for older people)
3. **Categorical Thresholds**: Capture step changes (e.g., obese BMI has extra cost beyond linear relationship)

**This is still Linear Regression** - we just created better features that help the model understand the data better. The model is now able to capture:
- Non-linear relationships through polynomial features
- Variable interactions through interaction terms
- Threshold effects through categorical features

**Result**: Higher accuracy without changing the algorithm!


## Step 6: Interpretation & Insights

Let's summarize what we've learned from our analysis and modeling.


### Key Findings from EDA

1. **Smokers have much higher charges**
   - The boxplot clearly showed smokers pay significantly more
   - This was confirmed by our statistical test (p-value < 0.05)
   - Smoking is the strongest predictor of insurance costs

2. **BMI strongly affects cost**
   - Higher BMI correlates with higher charges
   - This makes sense as BMI is linked to various health conditions
   - The relationship may be non-linear (very high BMI has disproportionate impact)

3. **Age has a positive linear trend**
   - Older individuals tend to have higher insurance charges
   - This is expected as age is associated with increased health risks
   - The relationship appears relatively consistent across age groups

### Key Modeling Insights

1. **Smoker variable has the strongest coefficient**
   - The model confirms smoking status is the most important factor
   - This aligns with our EDA and hypothesis testing results
   - Insurance companies should heavily weight smoking status in pricing

2. **BMI and Age are major predictors**
   - Both show significant positive coefficients
   - These demographic factors are crucial for accurate predictions
   - The model uses these to adjust base charges

3. **Region has little impact**
   - Regional coefficients are relatively small
   - Geographic location doesn't strongly influence charges in this dataset
   - This might vary in real-world scenarios with different healthcare systems


---

## Saving the Best Model

Let's save the best performing model so we can use it later for predictions.


In [ ]:
# Saving the best model to a .pkl file
import pickle
import os

# Determine which model performed best (comparing basic vs improved only)
best_model = None
best_model_name = ""
best_r2_final = max(r2, r2_improved)

if best_r2_final == r2_improved:
    best_model = model_improved
    best_model_name = "improved_model"
    print(f"✅ Best model: Improved with Feature Engineering (R² = {r2_improved:.4f})")
    print("Saving improved model...")
else:
    best_model = model
    best_model_name = "basic_model"
    print(f"✅ Best model: Basic Model (R² = {r2:.4f})")
    print("Saving basic model...")

# Save the model
model_filename = 'insurance_cost_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"\n✅ Model saved successfully as '{model_filename}'")

# Also save information about which model type and feature engineering
model_info = {
    'model_type': best_model_name,
    'r2_score': best_r2_final,
    'uses_feature_engineering': best_model_name == 'improved_model',
    'uses_log_transform': False,  # Not using log transform
    'feature_names': list(X_improved.columns) if best_model_name == 'improved_model' else list(X.columns)
}

with open('model_info.pkl', 'wb') as file:
    pickle.dump(model_info, file)

print(f"✅ Model info saved as 'model_info.pkl'")
print(f"\nModel Details:")
print(f"- Type: {model_info['model_type']}")
print(f"- R² Score: {model_info['r2_score']:.4f}")
print(f"- Uses Feature Engineering: {model_info['uses_feature_engineering']}")
print(f"- Uses Log Transform: {model_info['uses_log_transform']}")
print(f"- Number of Features: {len(model_info['feature_names'])}")


**Model Saved Successfully!**

The model is now saved and ready to be used in the Streamlit app. The saved model includes:
- The trained Linear Regression model
- Model metadata (type, R² score, feature names)
- Information about preprocessing steps needed

Next: We'll create a Streamlit app to use this model for predictions.


### How Hypothesis Testing Supported Findings

Our statistical test provided strong evidence (p-value < 0.05) that smokers have significantly higher mean charges than non-smokers. This quantitative confirmation:
- Validated our visual observations from EDA
- Gave us confidence in the model's emphasis on smoking status
- Provided a scientific basis for pricing decisions
- Showed the difference is not due to random chance

The confidence interval gave us a range for the true difference, making our conclusions more robust and actionable.


### What Could Be Improved?

**More Data:**
- Larger sample size would improve model reliability
- More diverse demographics would enhance generalizability
- Temporal data could capture trends over time

**More Features:**
- Medical history (pre-existing conditions)
- Lifestyle factors (exercise, diet)
- Family medical history
- Occupation and income levels
- Previous claims history

**Non-linear Models:**
- Polynomial regression for non-linear relationships
- Random Forest for complex interactions
- Gradient Boosting for better accuracy
- Neural networks for deep pattern recognition

These improvements could potentially increase R² score and reduce prediction errors.


## Step 7: Visualization & Presentation

All visualizations in this notebook have been created with:
- Clear titles and axis labels
- Appropriate color schemes
- Readable font sizes
- Grid lines for easier reading
- Captions and explanations

The plots use Seaborn and Matplotlib for professional, publication-ready figures.


## Step 8: Conclusion

### Summary of Findings

Our analysis revealed that **smoking status** is the strongest predictor of insurance charges, with smokers paying significantly more than non-smokers. **Age** and **BMI** also play important roles, showing positive relationships with insurance costs. The Linear Regression model achieved reasonable performance, explaining a substantial portion of variance in charges.

### Real-World Implications

Insurance companies can use these insights to:
- Develop risk-based pricing models
- Encourage smoking cessation programs (reduces costs)
- Target preventive care for high-BMI individuals
- Set age-appropriate premium structures

Policyholders can understand:
- How their lifestyle choices (smoking) impact costs
- The financial benefits of maintaining healthy BMI
- Expected cost increases with age

### Limitations

1. **Dataset size**: Limited sample may not represent all populations
2. **Feature availability**: Missing important factors like medical history
3. **Model simplicity**: Linear Regression assumes linear relationships
4. **Temporal factors**: No time-based trends or inflation adjustments
5. **Geographic specificity**: Results may vary by country/healthcare system

### Future Improvements

1. Collect more comprehensive data (medical history, lifestyle details)
2. Experiment with non-linear models (Random Forest, XGBoost)
3. Include interaction terms (e.g., BMI × Age, Smoker × Age)
4. Regular model updates as new data becomes available
5. Feature engineering to create more predictive variables

### Connecting EDA, Statistics, and Modeling

Our journey from EDA to modeling created a complete picture: **EDA** showed us visual patterns (smokers pay more), **statistical testing** confirmed these patterns with scientific rigor (p-value < 0.05), and **modeling** quantified the relationships (coefficients showing exact impact). Together, these three approaches validate each other and provide actionable insights for insurance pricing and policy decisions.
